# Code for plotting features for validity analysis
I'm not certain that the features I've developed are doing the job I intend them to do.

This code will take in the engineered feature set and output selected signals onto a chart.

In [1]:
#Import modules
import pandas as pd
import numpy as np
from rf_modules import *
import seaborn as sns
import matplotlib.pyplot as plt
import mpl_finance as fin
import matplotlib.dates as mdates
from matplotlib.dates import MONDAY, DateFormatter, DayLocator, WeekdayLocator
%matplotlib inline

In [14]:
#Import the orig data 
path = r"C:\Users\Robert\Documents\python_scripts\stock_trading_ml_modelling\historical_prices"
orig_df = pd.read_hdf(path + r"\all_hist_prices_w.h5")

In [15]:
#Import the data 
df_ft = pd.read_hdf(path + r"\all_hist_prices_w_ft_eng2.h5")

In [16]:
for c in df_ft:
    print(c)

ticker
date
open
close
high
low
volume
change_price
per_change_price
ema26
macd
signal_line
macd_line
close_orig_per_change_max_4
close_orig_per_change_max_13
close_orig_per_change_max_26
close_orig_per_change_max_52
close_orig_per_change_min_4
close_orig_per_change_min_13
close_orig_per_change_min_26
close_orig_per_change_min_52
macd_line_per_change_max_4
macd_line_per_change_max_13
macd_line_per_change_max_26
macd_line_per_change_max_52
macd_line_per_change_min_4
macd_line_per_change_min_13
macd_line_per_change_min_26
macd_line_per_change_min_52
change_close_shift1
close_max
close_min
prev_max_close
prev_min_close
prev_max_close_date_change
prev_min_close_date_change
max_change_close
min_change_close
prev_max_grad_close
prev_min_grad_close
prev_grad_conv_close
max_move_cum_close
min_move_cum_close
long_prev_max_move_date_close
long_prev_min_move_date_close
long_max_grad_close
long_min_grad_close
long_grad_conv_close
change_macd_shift1
macd_max
macd_min
prev_max_macd
prev_min_macd
pre

In [17]:
#Pick a share
# chart_df = df_ft[(df_ft.ticker == 'GFRD') & (df_ft.date >= '2014-01-01')]
# orig_chart_df = orig_df[(orig_df.ticker == 'GFRD') & (orig_df.date >= '2014-01-01')]
chart_df = df_ft[(df_ft.ticker == 'GFRD')]
orig_chart_df = orig_df[(orig_df.ticker == 'GFRD')]

In [32]:
chart_df[['date','close','long_max_grad_close','long_min_grad_close','long_grad_conv_close']]

,date,close,long_max_grad_close,long_min_grad_close,long_grad_conv_close
96129,1991-01-14,NaN,NaN,NaN,NaN
96130,1991-01-21,NaN,NaN,NaN,NaN
96131,1991-01-28,1.000000,NaN,NaN,NaN
96132,1991-02-04,1.000000,NaN,NaN,NaN
96133,1991-02-11,1.000000,NaN,NaN,NaN
96134,1991-02-18,1.000000,NaN,NaN,NaN
96135,1991-02-25,1.000000,NaN,NaN,NaN
96136,1991-03-04,1.000000,NaN,NaN,NaN
96137,1991-03-11,0.968762,NaN,NaN,NaN
96138,1991-03-18,0.906286,NaN,NaN,NaN


In [29]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2
    ,cols=1
#     ,vertical_spacing=0.3
    ,specs=[[{"secondary_y": True}]
            ,[{"secondary_y": True}]
#             ,[{"secondary_y": False}]
#             ,[{"secondary_y": False}]
           ]
)



###########NORMALISED PRICES###########
#Create graphs for normalised prices
fig.add_trace(go.Candlestick(x=chart_df.date
        ,open=chart_df.open
        ,high=chart_df.high
        ,low=chart_df.low
        ,close=chart_df.close
        ,name='OHLC')
    ,row=1,col=1
    ,secondary_y=False
)

def add_macd(_df,_row):
    #Create a graph for MACD
    pos_macd = _df.macd.copy()
    pos_macd.loc[pos_macd < 0] = 0
    neg_macd = _df.macd.copy()
    neg_macd.loc[neg_macd > 0] = 0
    fig.add_trace(
        go.Bar(
            x=_df.date
            ,y=pos_macd
            ,marker_color='green'
            ,name='Positive MACD'
        )
        ,row=_row,col=1
    )
    fig.add_trace(
        go.Bar(
            x=_df.date
            ,y=neg_macd
            ,marker_color='red'
            ,name='Negative MACD'
        )
        ,row=_row,col=1
    )
    fig.add_trace(go.Scatter(mode='lines'
            ,x=_df.date
            ,y=_df.macd_line
            ,name='MACD line')
        ,row=_row,col=1
    )
    fig.add_trace(
        go.Scatter(
            mode='lines'
            ,x=_df.date
            ,y=_df.signal_line
            ,name='Signal line')
        ,row=_row,col=1
    )
# add_macd(chart_df,2)



###########EVALUATE OTHER SIGNALS###########
def eval_signal(_signal,_row:int=1,_sec_y:bool=True):
    fig.add_trace(
        go.Scatter(
            mode='lines'
            ,x=chart_df.date
            ,y=chart_df[_signal]
            ,name=_signal
        )
        ,row=_row,col=1
        ,secondary_y=_sec_y
    )
eval_signal('change_close_shift1',_row=2,_sec_y=False)
eval_signal('close_orig_per_change_min_4',_row=2,_sec_y=True)

###########STANDARD PRICES###########
# #Create graphs for standard prices
# fig.add_trace(go.Candlestick(x=orig_chart_df.date
#         ,open=orig_chart_df.open
#         ,high=orig_chart_df.high
#         ,low=orig_chart_df.low
#         ,close=orig_chart_df.close
#         ,name='OHLC')
#     ,row=3,col=1
# )
# fig.add_trace(go.Scatter(mode='lines'
#         ,x=orig_chart_df.date
#         ,y=orig_chart_df.ema26
#         ,name='EMA26')
#     ,row=3,col=1
# )
# add_macd(orig_chart_df,4)

#Create a dictionary of shapes for buy and sell signals
shapes_li = []
def add_signal(sig_li,out_li:list=[],color:str='red',min_max:tuple=(0,1),ax:tuple=('x1','y1')):
    for date in sig_li:
        out_li.append({'type':'rect'
            ,'x0':date
            ,'x1':date
            ,'y0':min_max[0]
            ,'y1':min_max[1]
            ,'line':{'color':color}
            ,'opacity':0.4
            ,'xref':ax[0]
            ,'yref':ax[1]})
    return out_li
#Buy and sell
add_signal(
    chart_df[chart_df.signal == 'buy'].date.tolist()
    ,out_li=shapes_li
    ,color='green'
    ,min_max=(chart_df.low.min(),chart_df.high.max())
)
add_signal(
    chart_df[chart_df.signal == 'sell'].date.tolist()
    ,out_li=shapes_li
    ,color='red'
    ,min_max=(chart_df.low.min(),chart_df.high.max())
)
#Max and mins
def bool_signal(max_col,min_col,ref_col:str='close'):
    add_signal(
        chart_df[chart_df[max_col]].date.tolist()
        ,out_li=shapes_li
        ,color='green'
        ,min_max=(chart_df[ref_col].min(),chart_df[ref_col].max())
        ,ax=('x2','y3')
    )
    add_signal(
        chart_df[chart_df[min_col]].date.tolist()
        ,out_li=shapes_li
        ,color='red'
        ,min_max=(chart_df[ref_col].min(),chart_df[ref_col].max())
        ,ax=('x2','y3')
    )
# bool_signal('macd_max','macd_min','macd')



###########LAYOUT###########
#Update the layout
fig.update_layout(
    autosize=False
    ,width=1000
    ,height=800
)

#Establish range selector and buttons
rng_sel_di = dict(
    buttons=list([
        dict(count=1,
             label="1m",
             step="month",
             stepmode="backward"),
        dict(count=6,
             label="6m",
             step="month",
             stepmode="backward"),
        dict(count=1,
             label="YTD",
             step="year",
             stepmode="todate"),
        dict(count=1,
             label="1y",
             step="year",
             stepmode="backward"),
        dict(count=5,
             label="5y",
             step="year",
             stepmode="backward"),
        dict(count=3,
             label="3y",
             step="year",
             stepmode="backward"),
        dict(step="all")
    ])
)
for axis in ['xaxis'
             ,'xaxis2'
#              ,'xaxis3'
#              ,'xaxis4'
            ]:
    fig.layout[axis].rangeselector=rng_sel_di
    fig.layout[axis].rangeslider.visible=False
# fig.layout.yaxis.domain = [0.7,1.0]
# fig.layout.yaxis2.domain = [0.0,0.3]
fig.update_yaxes(automargin=True)
# print(fig)
fig.layout.shapes = shapes_li

fig.show()

C:\Users\Robert\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [7]:
#Function to find last max and mins
def prev_value(_df_in,_var_col,_bool_col,_gap:int=0):
    """Function to find last max and mins
    
    args:
    -----
    _df_in - pandas dataframe - must contain 'date', _var_col and _bool_col as column names
    _var_col - str - the name of the column containing the current variables
    _bool_col - str - the name of the column containing the bool values defining max and min vlaues
    _gap - int:0 - the number of period which must have elapsed before a min is 
        identified (prevents changing of min_flags on current week vs same week next week)
    
    returns:
    ------
    tuple - pandas series,pandas series - last max/min value, last max/min date
    """
    _df_in["prev_val"] = _df_in.loc[_df_in[_bool_col].shift(-_gap).fillna(False),_var_col]
    _df_in["prev_val"] = _df_in["prev_val"].fillna(method='ffill')
    _df_in["prev_marker_date"] = _df_in.loc[_df_in[_bool_col].shift(-_gap).fillna(False),"date"]
    _df_in["prev_marker_date"] = _df_in["prev_marker_date"].fillna(method='ffill')
    return (_df_in["prev_val"],_df_in["prev_marker_date"])

In [28]:
tmp_df.isna().sum().sort_values()

ticker                              0
long_prev_min_move_date_ema26       0
long_prev_max_move_date_ema26       0
min_move_cum_ema26                  0
max_move_cum_ema26                  0
                                 ... 
close_orig_per_change_max_13     1472
close_orig_per_change_max_4      1472
macd_line_per_change_max_52      1472
macd_line_per_change_max_4       1472
close_orig_per_change_min_26     1472
Length: 144, dtype: int64

In [26]:
tmp_df = chart_df.copy()
# tmp_df['prev_max_macd'],tmp_df['prev_max_macd_date_change'] = prev_value(tmp_df,'macd','macd_max',1)
tmp_df[tmp_df.date > '2018-12-31'][['ticker','date','macd','macd_max','prev_max_macd','prev_max_macd_date_change','max_change_macd','prev_max_grad_macd','close_orig_per_change_min_4']]

,ticker,date,macd,macd_max,prev_max_macd,prev_max_macd_date_change,max_change_macd,prev_max_grad_macd,close_orig_per_change_min_4
97552,GFRD,2019-01-07,-0.038322,False,0.014029,11.0,-0.052351,-0.001503,NaN
97553,GFRD,2019-01-14,-0.029167,False,0.014029,12.0,-0.043196,-0.001503,NaN
97554,GFRD,2019-01-21,-0.020418,False,0.014029,13.0,-0.034448,-0.001503,NaN
97555,GFRD,2019-01-28,-0.014244,False,0.014029,14.0,-0.028273,-0.001503,NaN
97556,GFRD,2019-02-04,-0.008470,False,0.014029,15.0,-0.022499,-0.001503,NaN
97557,GFRD,2019-02-11,0.000414,False,0.014029,16.0,-0.013615,-0.001503,NaN
97558,GFRD,2019-02-18,0.009959,False,0.014029,17.0,-0.004071,-0.001503,NaN
97559,GFRD,2019-02-25,0.026668,False,0.014029,18.0,0.012639,-0.001503,NaN
97560,GFRD,2019-03-04,0.034743,False,0.014029,19.0,0.020714,-0.001503,NaN
97561,GFRD,2019-03-11,0.044608,False,0.014029,20.0,0.030579,-0.001503,NaN
